In [1]:
# Necessary imports
import nltk
import glob
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [8]:
pos_files = glob.glob("data/clean_positive_train_0.csv")
neg_files = glob.glob("data/clean_negative_train_0.csv")

df_pos_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in pos_files[:3]]
df_neg_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in neg_files[:3]]

Now we make features for each dataset that we have, calculating all these features takes a long long time...

In [9]:
df = [pd.concat([df_pos, df_neg]) for (df_pos, df_neg) in zip(df_pos_list, df_neg_list)]

# Now we have scrambeled dataframes of 20000 entries, with features such as binary scores and profanity
df[0].describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality,sentiment,profanity,profanity_prob
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04
mean,0.500000,1127.415600,0.000050,0.973000,1347.648050,0.000350,0.033150,0.197250,2.679215e-01
std,0.500013,1375.725312,0.007071,0.162087,1465.051101,0.018705,0.283466,0.397933,2.922975e-01
min,0.000000,-2946.000000,0.000000,0.000000,-8907.000000,0.000000,-1.000000,0.000000,1.178028e-22
25%,0.000000,-129.000000,0.000000,1.000000,117.000000,0.000000,-0.037500,0.000000,8.408572e-02
50%,0.500000,820.000000,0.000000,1.000000,783.000000,0.000000,0.000000,0.000000,1.238206e-01
75%,1.000000,2183.000000,0.000000,1.000000,2329.000000,0.000000,0.160000,0.000000,3.362519e-01
max,1.000000,9582.000000,1.000000,1.000000,14097.000000,1.000000,1.000000,1.000000,1.000000e+00


In [10]:
df[0].head()

,text,score,ups,controversiality,parent_text,parent_score,parent_ups,parent_controversiality,sentiment,profanity,profanity_prob
0,twitter try comment article current activities,1,9582,0,fucking faggot,0,-7526,0,0.000000,0,0.121710
1,well exactly sounds like shoebox least wheneve...,1,9531,0,elaborate cum box please,1,3841,0,0.010472,1,0.999996
2,soviet russia bomb disarms,1,8545,0,dont live russia anymore going back time soon ...,1,621,0,0.000000,0,0.084165
3,runin senitur yolo,1,7430,0,made realize future presidents probably facebo...,1,4651,0,0.000000,0,0.120956
4,step motherfucker,1,7173,0,sex step mom dad around junior high school goi...,1,4251,0,0.000000,1,0.658445


In [41]:
tokenizer = Tokenizer(num_words=10000, lower=True, split=' ', document_count=0)
# Create the word_index list based on all our data",
text_data = [np.array2string(df[single_df]['text'].values.astype(str)) for single_df in range(len(df))]
text_data = ' '.join(text_data)
tokenizer.fit_on_texts(text_data)

In [ ]:
feature_data = np.array([])
score_data = []

X = []
Y = []

for n in range(len(df)):
    cur_df = df[n]
    df[n].dropna(axis=0, inplace=True)
    for index, row in cur_df.iterrows():
        sentiment = row['sentiment']
        profanity = row['profanity_prob']
        features = np.hstack((sentiment, profanity))
        print(features)

print(feature_data)
print(len(score_data))

[0.         0.12171034]
[0.01047237 0.9999965 ]
[0.        0.0841647]
[0.         0.12095604]
[0.         0.65844509]
[0.         0.20522307]
[-0.4         0.85044391]
[-0.33333333  0.1203524 ]
[-0.05916667  0.93541063]
[0.2        0.23758124]
[-1.          0.20862714]
[0.         0.11107722]
[0.         0.16693089]
[0.         0.06501211]
[0.         0.15931874]
[-0.18888889  0.03655717]
[0.3        0.10650104]
[0.         0.78869549]
[0.         0.29905719]
[0.         0.04108915]
[0.         0.12095604]
[0.         0.12095604]
[0.         0.08628473]
[0.         0.83529787]
[-0.03333333  0.22178997]
[0.         0.14376365]
[0.         0.13042687]
[0.         0.12203648]
[0.14285714 0.08998177]
[-0.09282407  0.99999993]
[0.         0.04714587]
[0.235      0.01858695]
[0.         0.12129133]
[0.125      0.02921326]
[0.375      0.02203386]
[0.27666667 0.00945083]
[0.         0.12210801]
[0.        0.1152176]
[-0.23333333  0.07826154]
[0.5        0.10164495]
[0.4        0.03917341]
[0. 

[0.303125   0.02040847]
[0.34642857 0.05319988]
[-0.03333333  0.38341751]
[0.         0.10326907]
[0.55       0.16730334]
[0.18988095 0.27705205]
[0.23333333 0.13571559]
[0.         0.08587995]
[0.         0.42462945]
[-0.2         0.10125641]
[0.11142857 0.91639719]
[-0.5         0.21613484]
[0.1        0.12625127]
[0.0952381  0.33854089]
[-0.39583333  0.04878429]
[0.5375     0.66854563]
[0.45       0.09490921]
[0.35       0.07834455]
[0.         0.07556665]
[0.6        0.06293637]
[-0.00625     0.14251683]
[0.         0.15846933]
[-0.15833333  0.78698254]
[0.125     0.1154904]
[0.25       0.03534126]
[0.7       0.1383807]
[-0.6         0.10394248]
[0.165625   0.30508268]
[0.         0.16522769]
[-0.16        0.13124976]
[0.01875    0.01781028]
[0.         0.23816914]
[0.525      0.18194258]
[0.         0.13964702]
[-0.04642857  0.59630324]
[-0.29166667  0.16466411]
[0.         0.08585728]
[0.25       0.08555839]
[0.         0.46266225]
[0.         0.08185827]
[0.09659091 0.08524717]


[0.25       0.78866638]
[0.         0.03137479]
[0.13333333 0.04076385]
[0.         0.19033711]
[0.65       0.19267597]
[0.03571429 0.88446733]
[0.5        0.03246456]
[0.56666667 0.02644096]
[0.29444444 0.06888405]
[-0.5         0.09602884]
[-0.05555556  0.99326719]
[0.         0.11038438]
[0.5       0.5972984]
[0.         0.08709775]
[0.        0.0780622]
[-0.7         0.12657969]
[0.         0.29032591]
[-0.7         0.81337988]
[0.         0.21751527]
[-0.14        0.14508041]
[0.         0.15415564]
[0.08333333 0.0537506 ]
[0.075      0.19811418]
[0.         0.12095604]
[-0.16666667  0.02262825]
[0.11666667 0.15908161]
[-0.2         0.88043993]
[0.         0.12367145]
[0.2        0.07120753]
[0.08333333 0.08107037]
[-0.2         0.08728942]
[0.         0.12412032]
[0.35       0.10027515]
[0.45       0.11710048]
[-0.2         0.11480836]
[0.         0.26523347]
[0.0952381  0.01000151]
[0.05       0.17771095]
[0.16666667 0.10529293]
[0.         0.20792505]
[0.3        0.13162261]
[0

[0.4       0.2174132]
[0.22777778 0.21301346]
[0.        0.1384521]
[-0.29166667  0.40045547]
[0.45       0.02818248]
[0.5        0.09804381]
[0.20097403 0.17562714]
[-0.22        0.08533308]
[0.13636364 0.17743865]
[0.00438312 0.00963878]
[0.5        0.10679199]
[0.41666667 0.373511  ]
[-0.05        0.00104811]
[-0.34444444  0.99871463]
[0.5        0.03527379]
[0.        0.0268147]
[-0.05        0.16887795]
[0.         0.42056954]
[4.58553792e-02 6.08428399e-06]
[0.2        0.05693102]
[0.4       0.0988349]
[0.14285714 0.06058828]
[0.         0.09877574]
[-0.13333333  0.74077342]
[0.         0.08556755]
[0.5        0.17417085]
[-0.16766513  1.        ]
[2.47744361e-02 2.53422590e-05]
[0.5        0.06416076]
[0.2        0.17848189]
[0.32666667 0.03393741]
[0.3        0.04506402]
[2.13970991e-01 3.55998194e-10]
[0.         0.11907477]
[0.         0.10702997]
[0.15178571 0.06008835]
[0.         0.09042698]
[0.         0.13808104]
[0.         0.11290532]
[-0.28333333  0.07780366]
[0.     

[-0.13511905  0.98840541]
[0.         0.17153025]
[0.125      0.07406003]
[0.         0.14238999]
[0.09375    0.01449272]
[-0.01944444  0.0897881 ]
[0.         0.10733809]
[0.         0.14218289]
[-0.128125    0.74118643]
[0.26666667 0.12503146]
[-0.275       0.12972095]
[-0.4         0.13140395]
[0.         0.10773068]
[-0.23809524  0.87779514]
[0.         0.12761218]
[0.5        0.13205088]
[-0.15833333  0.08722461]
[0.         0.26031466]
[-0.0875      0.00804538]
[0.         0.47115385]
[1.         0.06380529]
[0.14285714 0.11542904]
[-0.35        0.94799619]
[0.         0.13386081]
[1.09832766e-01 1.93750074e-05]
[0.1       0.1784829]
[0.         0.12080618]
[-0.125       0.09912365]
[0.18666667 0.05248118]
[0.         0.16711483]
[-0.1         0.01671127]
[0.         0.12378579]
[0.4        0.01774308]
[0.43       0.05269028]
[0.09166667 0.23771799]
[0.03333333 0.11399287]
[0.5        0.16845034]
[-0.03333333  0.04133775]
[-0.0375      0.28880475]
[0.1        0.03598385]
[0.     

In [ ]:
X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = train_test_split(feature_data[n], text_score[n], test_size=0.20, random_state=42)
X_train.append(X_train_tmp)
X_test.append(X_test_tmp)
y_train.append(y_train_tmp)
y_test.append(y_test_tmp)

X_train[n] = tf.keras.utils.normalize(X_train[n], axis=1)
X_test[n] = tf.keras.utils.normalize(X_test[n], axis=1)

## Finally we can do some neural networks!

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(24, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(16, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
train_data = X_train[0]

test_data = X_test[0]

history = model.fit(train_data,
                    y_train[0],
                    epochs=40,
                    validation_data=(test_data, y_test[0]),
                    verbose=1)